In [43]:
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.io import output_file, show
import pandas as pd
import numpy as np

from bokeh.plotting import figure

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (14,10)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
#Settings for plotting
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)

%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

In [44]:
datafilename = '../model_results_spread_prediction.csv'
df = pd.read_csv(datafilename)

In [49]:
df.End =  pd.to_datetime(df.End,format='%Y-%m-%d')

In [96]:
# curdf = df[np.round(np.multiply(df.RT,10)) == 13]

minRT = 9 
maxRT = 17
allRTs = np.arange(minRT,maxRT)

curRT = 17
curdf = df[np.round(np.multiply(df.RT,10)) == curRT] 

allAges = curdf.Age.unique()[:-1]


fig,(ax1,ax2) = plt.subplots(2,1)



agedf = curdf[curdf.Age == '10-19']
allDates = agedf.End
sumVals = np.zeros(agedf.Count[1:].shape)
# sumVals

for curAge in allAges:
    agedf = curdf[curdf.Age == curAge]

    # ax1.plot(agedf.End,agedf.HospCumul,'.-',label=curAge)
    # ax1.plot(agedf.End[1:],np.diff(agedf.HospCumul),'.-',label=curAge)

    sumVals = np.add(sumVals,np.diff(agedf.HospCumul))

# ax1.plot(allDates[1:],sumVals,'k')


thisSum = np.zeros(sumVals.shape)

for curAge in allAges:
    agedf = curdf[curdf.Age == curAge]

    prevSum = thisSum
    thisSum  = np.add(thisSum,np.diff(agedf.HospCumul))
    # ax1.plot(agedf.End,agedf.HospCumul,'.-',label=curAge)
    # ax2.plot(agedf.End[1:],np.divide(thisSum,sumVals),'.-',label=curAge)
    ax2.fill_between(agedf.End[1:],np.divide(thisSum,sumVals),np.divide(prevSum,sumVals),'.-',label=curAge)
    ax1.fill_between(agedf.End[1:],thisSum,prevSum,'.-',label=curAge)


# plt.figure()
# plt.plot(curdf.End,curdf.Count,'.')
ax1.legend()
ax2.legend()

ax1.set_ylim(bottom=0)
ax2.set_ylim(bottom=0)
ax2.set_ylim([0,1])
plt.tight_layout()

plt.savefig('testStack'+str(curRT))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-96-a1811862037b>:42: MatplotlibDeprecationWarning: Since 3.2, the parameter *where* must have the same size as [18765.0 18772.0 18779.0 18786.0 18793.0 18800.0 18807.0 18814.0 18821.0
 18828.0 18835.0 18842.0 18849.0 18856.0 18863.0 18870.0 18877.0 18884.0
 18891.0 18898.0 18905.0] in fill_betweenx(). This will become an error two minor releases later.
  ax2.fill_between(agedf.End[1:],np.divide(thisSum,sumVals),np.divide(prevSum,sumVals),'.-',label=curAge)
<ipython-input-96-a1811862037b>:43: MatplotlibDeprecationWarning: Since 3.2, the parameter *where* must have the same size as [18765.0 18772.0 18779.0 18786.0 18793.0 18800.0 18807.0 18814.0 18821.0
 18828.0 18835.0 18842.0 18849.0 18856.0 18863.0 18870.0 18877.0 18884.0
 18891.0 18898.0 18905.0] in fill_betweenx(). This will become an error two minor releases later.
  ax1.fill_between(agedf.End[1:],thisSum,prevSum,'.-',label=curAge)


In [ ]:

x = [x*0.005 for x in range(0, 201)]

source = ColumnDataSource(data=dict(x=x, y=x))

plot = figure(plot_width=400, plot_height=400)
plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

slider = Slider(start=0.1, end=6, value=1, step=.1, title="power")

update_curve = CustomJS(args=dict(source=source, slider=slider), code="""
    var data = source.data;
    var f = slider.value;
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.pow(x[i], f)
    }
    
    // necessary becasue we mutated source.data in-place
    source.change.emit();
""")
slider.js_on_change('value', update_curve)

output_file("plot.html")

show(column(slider, plot))